In [1]:
from pydub import AudioSegment
import os
import math
from transformers import pipeline
import gradio as gr
from docx import Document
import torch
import requests
import ollama

In [2]:
def split_audio(input_file, output_folder, duration):
    audio = AudioSegment.from_mp3(input_file)
    total_length = len(audio)
    num_parts = math.ceil(total_length / (duration * 1000))

    for i in range(num_parts):
        start = i * duration * 1000
        end = (i + 1) * duration * 1000
        split_audio = audio[start:end]
        output_path = os.path.join(output_folder, f"part_{i+1}.mp3")
        split_audio.export(output_path, format="mp3")
        print(f"Exported {output_path}")

In [38]:
import requests
from docx import Document

def censurar_docx(file_path, output_path):
    
    doc = Document(file_path)
    
    def censurar_texto(texto):
        try:
            messages = [{"role": "user", "content" : "Substitute the obscene words in this text with ****, in case that there are no obscene words, return the document as it is. Answer only with the censored text,don't add anything else. And this is the text: " + texto}]
            response = ollama.chat(model="llama3.2", messages=messages)
            return response["message"]["content"]
        except requests.RequestException as e:
            print(f"Error al conectarse a Ollama: {e}")
            return texto

    for paragraph in doc.paragraphs:
        original_text = paragraph.text
        if original_text.strip():
            censored_text = censurar_texto(original_text)
            paragraph.text = censored_text
    
    doc.save(output_path)
    print(f"Documento censurado guardado en: {output_path}")


In [32]:
censurar_docx("transcripcion1.docx", "censurado.docx")

 1, 2, 3! 1, 2, 3! It's a very good way to, very good way to start. 1, 2, 3! It's a very good way, it's a very good way to start. 4, 5, 6! It's a very good way to, very good way to start. 4, 5, 6! It's a very good way to, very good way to start. 1, 2, 3! 4, 5, 6! It's a very good way to start. 1, 2, 3, 4, 5, 6, it's a very good way to start These are the numbers we all know We'll learn more as we grow the numbers It wouldn't be even if we didn't have 7 It's a very good place to start It wouldn't be even if we didn't have 7 It's a very good place to start 7 and 8 and 9 and 10 It's a very good place to start 7 and 8 and 9 and 10 It's a very good place to start 1, 2, 3, 4, 5, 6 It's a very good place to start 7 and 8 and 9 and 10 It's a very good place to start These are the numbers we all know We'll learn more as we grow the numbers These are the numbers we all know Berlin, Borg, A.G.O. and Nobles fuck you bitch
Documento censurado guardado en: censurado.docx


In [39]:
def transcribir_con_censura(file):
  device = 0 if torch.cuda.is_available() else -1
  pipe = pipeline("automatic-speech-recognition", "openai/whisper-large-v3", return_timestamps=True, device=device)
  output_dir = "Salida"
  os.makedirs(output_dir, exist_ok=True)
  split_audio(file, output_dir, 900)
  doc = Document()

  for file in sorted(os.listdir(output_dir)):
      file_path = os.path.join(output_dir, file)
      print(f"Procesando: {file_path}")

      transcription = pipe(file_path)["text"]
      doc.add_paragraph(transcription)

  output_file = "transcripcion.docx"
  doc.save(output_file)

  doc1 = "censurado.docx"

  censurar_docx(output_file, doc1)

  return doc1

In [34]:
def transcribir_sin_censura(file):
  device = 0 if torch.cuda.is_available() else -1
  pipe = pipeline("automatic-speech-recognition", "openai/whisper-large-v3", return_timestamps=True, device=device)
  output_dir = "Salida"
  os.makedirs(output_dir, exist_ok=True)
  split_audio(file, output_dir, 900)
  doc = Document()

  for file in sorted(os.listdir(output_dir)):
      file_path = os.path.join(output_dir, file)
      print(f"Procesando: {file_path}")

      transcription = pipe(file_path)["text"]
      doc.add_paragraph(transcription)

  output_file = "transcripcion.docx"
  doc.save(output_file)

  return output_file

In [11]:
demo = gr.Blocks()

with demo:
    audio_file = gr.Audio(type="filepath")

    b1 = gr.Button("Con censura")
    b2 = gr.Button("Sin censura")

    b1.click(transcribir_con_censura, inputs=audio_file, outputs=gr.File())
    b2.click(transcribir_sin_censura, inputs=audio_file, outputs=gr.File())

demo.queue(default_concurrency_limit=10).launch(share=True,debug=True)

* Running on local URL:  http://127.0.0.1:7860

Could not create share link. Please check your internet connection or our status page: https://status.gradio.app.


Exported Salida\part_1.mp3
Procesando: Salida\part_1.mp3


c:\Users\OscarZ\Desktop\Todo\Estudio\Oidor\.venv\Lib\site-packages\transformers\models\whisper\generation_whisper.py:509: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(


Keyboard interruption in main thread... closing server.


In [40]:
demo = gr.Interface(
    fn=transcribir_con_censura,
    inputs=gr.File(),
    outputs=gr.File(),
    allow_flagging='never'
)
demo.launch(share=True,debug=True)

c:\Users\OscarZ\Desktop\Todo\Estudio\Oidor\.venv\Lib\site-packages\gradio\interface.py:399: UserWarning: The `allow_flagging` parameter in `Interface` is deprecated.Use `flagging_mode` instead.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7860

Could not create share link. Please check your internet connection or our status page: https://status.gradio.app.


Exported Salida\part_1.mp3
Procesando: Salida\part_1.mp3


c:\Users\OscarZ\Desktop\Todo\Estudio\Oidor\.venv\Lib\site-packages\transformers\models\whisper\generation_whisper.py:509: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(


Documento censurado guardado en: censurado.docx
Keyboard interruption in main thread... closing server.
